In [1]:
import os
import sys
import time

import numpy as np
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
tfb = tfp.bijectors
tfd = tfp.distributions
import matplotlib.pyplot as plt

from normalizingflows.flow_catalog import RealNVP
from utils.train_utils import train_density_estimation, nll
from data import dataset_loader




/Users/lenny/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensorflow:  2.4.1
tensorflow-probability:  0.12.2


In [2]:
category = 2
train_data, val_data, test_data, _ = dataset_loader.load_and_preprocess_mnist(logit_space=True, batch_size=128, shuffle=True, classes=category)

2022-08-16 12:08:04.669895: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-16 12:08:04.670391: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
mnist_shape = (28, 28, 1)
size = 28
input_shape = size*size
permutation = tf.cast(np.concatenate((np.arange(input_shape/2,input_shape),np.arange(0,input_shape/2))), tf.int32)
base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros(input_shape, tf.float32))

In [4]:
n_images = 1
dataset = "mnist"
exp_number = 1
max_epochs = 200
layers = 5
shape = [256, 256]
base_lr = 1e-4
end_lr = 1e-5

In [5]:
bijectors = []
alpha = 1e-3

for i in range(layers):
    bijectors.append(tfb.BatchNormalization())
    bijectors.append(RealNVP(input_shape=input_shape, n_hidden=shape))
    bijectors.append(tfp.bijectors.Permute(permutation))
    
bijectors.append(tfb.Reshape(event_shape_out=(size, size),
                                 event_shape_in=(size * size,)))


bijector = tfb.Chain(bijectors=list(reversed(bijectors)), name='chain_of_real_nvp')

flow = tfd.TransformedDistribution(
    distribution=base_dist,
    bijector=bijector
)

# number of trainable variables
n_trainable_variables = len(flow.trainable_variables)

2022-08-16 12:08:05.923315: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [6]:
learning_rate_fn = tf.keras.optimizers.schedules.PolynomialDecay(base_lr, max_epochs, end_lr, power=0.5)

checkpoint_directory = "{}/tmp_{}_{}_{}_{}_{}".format(dataset, layers, shape[0], shape[1], exp_number, category)
checkpoint_prefix = os.path.join(checkpoint_directory, "ckpt")

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
checkpoint = tf.train.Checkpoint(optimizer=opt, model=flow)

In [7]:
global_step = []
train_losses = []
val_losses = []
min_val_loss = tf.convert_to_tensor(np.inf, dtype=tf.float32)  # high value to ensure that first loss < min_loss
min_train_loss = tf.convert_to_tensor(np.inf, dtype=tf.float32)
min_val_epoch = 0
min_train_epoch = 0
delta_stop = 50  # threshold for early stopping

t_start = time.time()  # start time

# start training
for i in range(max_epochs):

    train_data.shuffle(buffer_size=1000)
    batch_train_losses = []
    for batch in train_data:
        batch_loss = train_density_estimation(flow, opt, batch)
        batch_train_losses.append(batch_loss)

    train_loss = tf.reduce_mean(batch_train_losses)

    if i % int(1) == 0:
        batch_val_losses = []
        for batch in val_data:
            batch_loss = nll(flow, batch)
            batch_val_losses.append(batch_loss)

        val_loss = tf.reduce_mean(batch_val_losses)

        global_step.append(i)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f"{i}, train_loss: {train_loss}, val_loss: {val_loss}")

        if train_loss < min_train_loss:
            min_train_loss = train_loss
            min_train_epoch = i

        if val_loss < min_val_loss:
            min_val_loss = val_loss
            min_val_epoch = i
            checkpoint.write(file_prefix=checkpoint_prefix)

        elif i - min_val_epoch > delta_stop:  # no decrease in min_val_loss for "delta_stop epochs"
            break

train_time = time.time() - t_start

# load best model with min validation loss
checkpoint.restore(checkpoint_prefix)

# perform on test dataset
t_start = time.time()

test_losses = []
for batch in test_data:
    batch_loss = nll(flow, batch)
    test_losses.append(batch_loss)

test_loss = tf.reduce_mean(test_losses)

test_time = time.time() - t_start

save_dir = "{}/sampling_{}_{}_{}_{}/".format(dataset, layers, shape[0], shape[1], category)

if not os.path.isdir(save_dir):
    os.mkdir(save_dir)
for j in range(n_images):
    plt.figure()
    data = flow.sample(1)
    data = dataset_loader.inverse_logit(data)
    data = tf.reshape(data, (1, size, size))
    plt.imshow(data[0], cmap='gray')
    plt.savefig("{}/{}_{}_i{}.png".format(save_dir, exp_number, min_val_epoch, j))
    plt.close()

# remove checkpoint
filelist = [f for f in os.listdir(checkpoint_directory)]
for f in filelist:
    os.remove(os.path.join(checkpoint_directory, f))
os.removedirs(checkpoint_directory)

print(f'Test loss: {test_loss} at epoch: {i}')
print(f'Average test log likelihood: {-test_loss} at epoch: {i}')
print(f'Min val loss: {min_val_loss} at epoch: {min_val_epoch}')
print(f'Last val loss: {val_loss} at epoch: {i}')
print(f'Min train loss: {min_train_loss} at epoch: {min_train_epoch}')
print(f'Last train loss: {train_loss} at epoch: {i}')
print(f'Training time: {train_time}')
print(f'Test time: {test_time}')

results = {
    'test_loss': float(test_loss),
    'avg_test_logll': float(-test_loss),
    'min_val_loss': float(min_val_loss),
    'min_val_epoch': min_val_epoch,
    'val_loss': float(val_loss),
    'min_train_loss': float(min_train_loss),
    'min_train_epoch': min_train_epoch,
    'train_loss': float(train_loss),
    'train_time': train_time,
    'test_time': test_time,
    'trained_epochs': i,
    'trainable variables': n_trainable_variables,
    'exp_number': exp_number
}

Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.


/Users/lenny/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:2273: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2022-08-16 12:08:09.995730: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


0, train_loss: 1325.5394287109375, val_loss: 1134.71533203125
1, train_loss: 1062.76416015625, val_loss: 965.9075927734375
2, train_loss: 914.4681396484375, val_loss: 837.2979736328125
3, train_loss: 787.5392456054688, val_loss: 716.618896484375
4, train_loss: 675.7529907226562, val_loss: 612.38525390625
5, train_loss: 598.2848510742188, val_loss: 580.1678466796875
6, train_loss: 572.3732299804688, val_loss: 553.2103271484375
7, train_loss: 539.8829956054688, val_loss: 526.51953125
8, train_loss: 515.4192504882812, val_loss: 500.64947509765625
9, train_loss: 480.37811279296875, val_loss: 475.1078186035156
10, train_loss: 443.8014221191406, val_loss: 449.7270202636719
11, train_loss: 414.1618957519531, val_loss: 424.8367004394531
12, train_loss: 379.9648742675781, val_loss: 400.7292175292969
13, train_loss: 351.6256408691406, val_loss: 377.64727783203125
14, train_loss: 317.39739990234375, val_loss: 355.55059814453125
15, train_loss: 283.036376953125, val_loss: 333.39678955078125
16, tr